In [48]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langchain.prompts import PromptTemplate
import os
from pydantic import BaseModel, Field
import operator
from typing import Annotated

In [49]:
load_dotenv()

os.environ["LANGCHAIN_TRACING_V2"] = os.getenv("LANGCHAIN_TRACING_V2", "true")
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT", "essay-evaluation-workflow")

if not os.getenv("LANGCHAIN_API_KEY"):
    raise ValueError("Missing LANGCHAIN_API_KEY for LangSmith tracing.")

print(f"LangSmith tracing enabled for project: {os.environ['LANGCHAIN_PROJECT']}")


LangSmith tracing enabled for project: essay-evaluation-workflow


In [50]:
model = ChatOpenAI(model="gpt-4o-mini")

In [51]:
model

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x10f6af2f0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x10f6ac4a0>, root_client=<openai.OpenAI object at 0x10f6a4f20>, root_async_client=<openai.AsyncOpenAI object at 0x10f6adcd0>, model_name='gpt-4o-mini', model_kwargs={}, openai_api_key=SecretStr('**********'))

In [52]:
class EvaluationSchema(BaseModel):
    score: int = Field(..., ge=1, le=10, description="Score between 1 and 10")
    feedback: str = Field(..., description="Detailed feedback on the essay")

In [53]:
structured_model = model.with_structured_output(EvaluationSchema)

In [54]:
essay = """
India in the Age of AI

As the world enters a transformative era defined by artificial intelligence (AI), India stands at a critical juncture — one where it can either emerge as a global leader in AI innovation or risk falling behind in the technology race. The age of AI brings with it immense promise as well as unprecedented challenges, and how India navigates this landscape will shape its socio-economic and geopolitical future.

India's strengths in the AI domain are rooted in its vast pool of skilled engineers, a thriving IT industry, and a growing startup ecosystem. With over 5 million STEM graduates annually and a burgeoning base of AI researchers, India possesses the intellectual capital required to build cutting-edge AI systems. Institutions like IITs, IIITs, and IISc have begun fostering AI research, while private players such as TCS, Infosys, and Wipro are integrating AI into their global services. In 2020, the government launched the National AI Strategy (AI for All) with a focus on inclusive growth, aiming to leverage AI in healthcare, agriculture, education, and smart mobility.

One of the most promising applications of AI in India lies in agriculture, where predictive analytics can guide farmers on optimal sowing times, weather forecasts, and pest control. In healthcare, AI-powered diagnostics can help address India’s doctor-patient ratio crisis, particularly in rural areas. Educational platforms are increasingly using AI to personalize learning paths, while smart governance tools are helping improve public service delivery and fraud detection.

However, the path to AI-led growth is riddled with challenges. Chief among them is the digital divide. While metropolitan cities may embrace AI-driven solutions, rural India continues to struggle with basic internet access and digital literacy. The risk of job displacement due to automation also looms large, especially for low-skilled workers. Without effective skilling and re-skilling programs, AI could exacerbate existing socio-economic inequalities.

Another pressing concern is data privacy and ethics. As AI systems rely heavily on vast datasets, ensuring that personal data is used transparently and responsibly becomes vital. India is still shaping its data protection laws, and in the absence of a strong regulatory framework, AI systems may risk misuse or bias.

To harness AI responsibly, India must adopt a multi-stakeholder approach involving the government, academia, industry, and civil society. Policies should promote open datasets, encourage responsible innovation, and ensure ethical AI practices. There is also a need for international collaboration, particularly with countries leading in AI research, to gain strategic advantage and ensure interoperability in global systems.

India’s demographic dividend, when paired with responsible AI adoption, can unlock massive economic growth, improve governance, and uplift marginalized communities. But this vision will only materialize if AI is seen not merely as a tool for automation, but as an enabler of human-centered development.

In conclusion, India in the age of AI is a story in the making — one of opportunity, responsibility, and transformation. The decisions we make today will not just determine India’s AI trajectory, but also its future as an inclusive, equitable, and innovation-driven society.
"""

In [55]:
prompt = f'Evaluate the language quality of the following essay and provide a feedback and assign a score out of 10 \n {essay}'

In [56]:
# Withoout structured output
model.invoke(prompt)

AIMessage(content='### Feedback on the Essay \n\n**Language Quality Evaluation:** \n\n1. **Clarity and Cohesion:** The essay presents a clear and well-structured argument regarding the opportunities and challenges that India faces in the realm of artificial intelligence. Each paragraph transitions smoothly into the next, which helps maintain the reader\'s engagement. Concepts are explained logically, resulting in a coherent overall argument.\n\n2. **Use of Evidence:** The essay effectively utilizes statistics and examples, such as India\'s annual STEM graduates and references to institutions like IITs, which bolster claims and provide concrete evidence to support arguments. Mentioning specific applications of AI in agriculture, healthcare, and education illustrates the practical implications of the topic well.\n\n3. **Grammar and Syntax:** The essay is largely free from grammatical errors. Sentence structure varies, which adds to the essay\'s readability. The use of complex sentences i

In [57]:
# With structured output
structured_model.invoke(prompt)

EvaluationSchema(score=9, feedback="The essay presents a coherent and insightful analysis of India's position in the age of AI, effectively highlighting both its strengths and challenges. The language quality is high, with clear, concise sentences and a good variety of vocabulary. The introduction draws in the reader by setting the context and posing a critical question, while the body of the essay is well-structured with logical flow and transitions between points.\n\nThe use of specific data, such as the number of STEM graduates, enhances the argument and provides credibility to the assertions made. Each application of AI discussed (in agriculture, healthcare, and education) is relevant and well-explained, illustrating significant real-world impacts. The recognition of challenges, such as the digital divide and data privacy, adds depth to the analysis, showing a nuanced understanding of the subject.\n\nImprovements could include a more detailed exploration of potential solutions to t

In [58]:
# Create essay class
class UPSCState(TypedDict):
    essay: str
    language_feedback: str
    analysis_feedback: str
    clarity_feedback: str
    overall_feedback: str
    individual_scores: Annotated[list[int], operator.add]
    avg_score: float

In [59]:
def evaluate_language(state: UPSCState):
    prompt = f'Evaluate the language quality of the following essay and provide a feedback and assign a score out of 10 \n {state["essay"]}'
    result = structured_model.invoke(prompt)
    return {"language_feedback": result.feedback, "individual_scores": [result.score]}

In [60]:
def evaluate_analysis(state: UPSCState):
    prompt = f'Evaluate the depth of analysis quality of the following essay and provide a feedback and assign a score out of 10 \n {state["essay"]}'
    result = structured_model.invoke(prompt)
    return {"analysis_feedback": result.feedback, "individual_scores": [result.score]}

In [61]:
def evaluate_clarity(state: UPSCState):
    prompt = f'Evaluate the clarity of thought of the following essay and provide a feedback and assign a score out of 10 \n {state["essay"]}'
    result = structured_model.invoke(prompt)
    return {"clarity_feedback": result.feedback, "individual_scores": [result.score]}

In [62]:
def overall_evaluation(state: UPSCState):

    # summary feedback
    prompt = f'Based on the following feedbacks create a summarized feedback \n language feedback - {state["language_feedback"]} \n depth of analysis feedback - {state["analysis_feedback"]} \n clarity of thought feedback - {state["clarity_feedback"]}'
    overall_feedback = model.invoke(prompt).content

    # avg calculate
    avg_score = sum(state['individual_scores'])/len(state['individual_scores'])

    return {'overall_feedback': overall_feedback, 'avg_score': avg_score}

In [63]:
# Create graph
graph = StateGraph(UPSCState)

# Add nodes
graph.add_node('evaluate_language', evaluate_language)
graph.add_node('evaluate_analysis', evaluate_analysis)
graph.add_node('evaluate_clarity', evaluate_clarity)
graph.add_node('overall_evaluation', overall_evaluation)

# Define edges
graph.add_edge(START, 'evaluate_language')
graph.add_edge(START, 'evaluate_analysis')
graph.add_edge(START, 'evaluate_clarity')

graph.add_edge('evaluate_language', 'overall_evaluation')
graph.add_edge('evaluate_analysis', 'overall_evaluation')
graph.add_edge('evaluate_clarity', 'overall_evaluation')

graph.add_edge('overall_evaluation', END)

# Compile graph
workflow = graph.compile().with_config({"run_name": "EssayEvaluationWorkflow"})


In [64]:
initial_state = {"essay": essay}

In [65]:
workflow.invoke(initial_state)

{'essay': "\nIndia in the Age of AI\n\nAs the world enters a transformative era defined by artificial intelligence (AI), India stands at a critical juncture — one where it can either emerge as a global leader in AI innovation or risk falling behind in the technology race. The age of AI brings with it immense promise as well as unprecedented challenges, and how India navigates this landscape will shape its socio-economic and geopolitical future.\n\nIndia's strengths in the AI domain are rooted in its vast pool of skilled engineers, a thriving IT industry, and a growing startup ecosystem. With over 5 million STEM graduates annually and a burgeoning base of AI researchers, India possesses the intellectual capital required to build cutting-edge AI systems. Institutions like IITs, IIITs, and IISc have begun fostering AI research, while private players such as TCS, Infosys, and Wipro are integrating AI into their global services. In 2020, the government launched the National AI Strategy (AI 

### Test with a bad essay

In [66]:
essay2 = """India and AI Time

Now world change very fast because new tech call Artificial Intel… something (AI). India also want become big in this AI thing. If work hard, India can go top. But if no careful, India go back.

India have many good. We have smart student, many engine-ear, and good IT peoples. Big company like TCS, Infosys, Wipro already use AI. Government also do program “AI for All”. It want AI in farm, doctor place, school and transport.

In farm, AI help farmer know when to put seed, when rain come, how stop bug. In health, AI help doctor see sick early. In school, AI help student learn good. Government office use AI to find bad people and work fast.

But problem come also. First is many villager no have phone or internet. So AI not help them. Second, many people lose job because AI and machine do work. Poor people get more bad.

One more big problem is privacy. AI need big big data. Who take care? India still make data rule. If no strong rule, AI do bad.

India must all people together – govern, school, company and normal people. We teach AI and make sure AI not bad. Also talk to other country and learn from them.

If India use AI good way, we become strong, help poor and make better life. But if only rich use AI, and poor no get, then big bad thing happen.

So, in short, AI time in India have many hope and many danger. We must go right road. AI must help all people, not only some. Then India grow big and world say "good job India"."""

In [67]:
initial_state = {"essay": essay2}

workflow.invoke(initial_state)

{'essay': 'India and AI Time\n\nNow world change very fast because new tech call Artificial Intel… something (AI). India also want become big in this AI thing. If work hard, India can go top. But if no careful, India go back.\n\nIndia have many good. We have smart student, many engine-ear, and good IT peoples. Big company like TCS, Infosys, Wipro already use AI. Government also do program “AI for All”. It want AI in farm, doctor place, school and transport.\n\nIn farm, AI help farmer know when to put seed, when rain come, how stop bug. In health, AI help doctor see sick early. In school, AI help student learn good. Government office use AI to find bad people and work fast.\n\nBut problem come also. First is many villager no have phone or internet. So AI not help them. Second, many people lose job because AI and machine do work. Poor people get more bad.\n\nOne more big problem is privacy. AI need big big data. Who take care? India still make data rule. If no strong rule, AI do bad.\n\n